### In Colab

In [1]:
from google.colab import drive
drive.mount('drive', force_remount=True)

Mounted at drive


In [2]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=bcccb85987f049640feaf2b4a14650dee4ff7608b1b9a61c0e33c8f2747f20e6
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


### Imports

In [3]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from seqeval.metrics import classification_report

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Read Dataset

In [4]:
with open('drive/MyDrive/Experiments/SNLP2/week_07/all_data.data', 'rt', encoding='utf8') as fr:
    all_data = fr.read().split('\n')
    all_labels = set()
    X, Y, xx, yy = [], [], [], []
    for line in all_data:
        if line.strip():
            w, label, _, _, _, _ = line.split('\t')
            all_labels.add(label)
            xx.append(w.lower())
            yy.append(label)
        else:
            X.append(xx.copy())
            Y.append(yy.copy())
            xx.clear()
            yy.clear()
assert len(X) == len(Y)
print(f'data documents: {len(X)}\n'
      f'sent: {X[0]}\n'
      f'labels: {Y[0]}')

data documents: 1312
sent: ['analysis', 'of', 'the', 'efficacy', 'of', 'diet', 'and', 'short-term', 'probiotic', 'intervention', 'on', 'depressive', 'symptoms', 'in', 'patients', 'after', 'bariatric', 'surgery', ':', 'a', 'randomized', 'double-blind', 'placebo', 'controlled', 'pilot', 'study', '.']
labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DDF', 'I-DDF', 'O', 'B-human', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [5]:
print(f'labels: {len(all_labels), all_labels}')

labels: (27, {'B-DDF', 'B-food', 'B-gene', 'B-human', 'B-anatomical%location', 'B-biomedical%technique', 'I-bacteria', 'B-animal', 'B-drug', 'I-human', 'B-statistical%technique', 'I-food', 'O', 'B-chemical', 'I-statistical%technique', 'B-microbiome', 'B-dietary%supplement', 'I-animal', 'I-microbiome', 'I-dietary%supplement', 'B-bacteria', 'I-DDF', 'I-gene', 'I-biomedical%technique', 'I-chemical', 'I-anatomical%location', 'I-drug'})


In [6]:
label2id = {label: i for i, label in enumerate(list(all_labels))}
id2label = {v: k for k, v in label2id.items()}

### Split into Train and Test

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

print(len(X_train), len(X_test))

1049 263


In [8]:
vocab = ['<UNK>'] + sorted({w for w in X_train for w in w})
vocab2id = {w: i for i, w in enumerate(vocab)}
id2vocab = {v: k for k, v in vocab2id.items()}

print(f'size of vocab: {len(vocab)}')

size of vocab: 4167


### Encode the input

In [9]:
X_train_tensors = [torch.tensor([vocab2id[w] for w in train_sent], dtype=torch.long) for train_sent in X_train]
X_test_tensors = [torch.tensor([vocab2id.get(w, vocab2id['<UNK>']) for w in test_sent], dtype=torch.long) for test_sent
                  in X_test]

print(len(X_train_tensors), len(X_test_tensors))

1049 263


In [10]:
Y_train_tensors = [torch.tensor([label2id[label] for label in train_y], dtype=torch.long) for train_y in Y_train]
Y_test_tensors = [torch.tensor([label2id[label] for label in test_y], dtype=torch.long) for test_y in Y_test]

print(len(Y_train_tensors), len(Y_test_tensors))

1049 263


### GRU Network, Loss Function and Optimizer

In [11]:
class Net(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, output_size, bidirectional, num_layers):
        super(Net, self).__init__()
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, bidirectional=bidirectional, num_layers=num_layers)
        self.clf1 = nn.Linear(hidden_size, output_size)

    def forward(self, X, h0):
        e = self.embedding(X)
        o, h0 = self.gru(e, h0)
        if self.bidirectional:
            # sum two directions
            forward = o[:, : o.shape[1] // 2]
            backward = o[:, o.shape[1] // 2:]
            o = forward + backward
        return self.clf1(o)

    def init_hidden(self):
        dims = 1 if not self.bidirectional else 2
        return torch.zeros(dims * self.num_layers, self.hidden_size)

In [52]:
### hyperparameters
input_size = len(vocab2id)
emb_size = 512
hidden_size = 2 * emb_size
output_size = len(label2id)
lr = 5e-5
epochs = 50
bidirectional = True
num_layers = 2
model_name = 'gru-2-bidirectional-50-epochs'

net = Net(input_size=input_size, emb_size=emb_size, hidden_size=hidden_size, output_size=output_size,
          bidirectional=bidirectional, num_layers=num_layers).to(device)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(net.parameters(), lr=lr)

### Training Loop

In [53]:
net.train()

for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    for i in range(len(X_train_tensors)):
        h0 = net.init_hidden().to(device)
        train_tensor, train_y = X_train_tensors[i].to(device), Y_train_tensors[i].to(device)
        opt.zero_grad()
        pred_y = net.forward(train_tensor, h0)
        loss = loss_fn(pred_y, train_y)
        loss.backward()
        opt.step()

        epoch_loss += loss.item()
    print(f'loss: {epoch_loss / len(X_train_tensors)}')

  2%|▏         | 1/50 [00:07<05:59,  7.35s/it]

loss: 0.7496948129576484


  4%|▍         | 2/50 [00:14<05:53,  7.37s/it]

loss: 0.39466586012439403


  6%|▌         | 3/50 [00:22<05:47,  7.39s/it]

...

loss: 3.2608907681972655e-06


 98%|█████████▊| 49/50 [06:00<00:07,  7.33s/it]

loss: 2.045058253977783e-06


100%|██████████| 50/50 [06:07<00:00,  7.35s/it]

loss: 1.2716008066697665e-06


### Inference

In [54]:
net.eval()

Y_pred = []
with torch.no_grad():
    for i in tqdm(range(len(X_test_tensors))):
        h0 = net.init_hidden().to(device)
        test_tensor, test_y = X_test_tensors[i].to(device), Y_test_tensors[i].to(device)
        pred_y = net.forward(test_tensor, h0)

        # print([id2label[_id_.item()] for _id_ in torch.argmax(pred_y, dim=1)])
        Y_pred.append([id2label[_id_.item()] for _id_ in torch.argmax(pred_y, dim=1)])


100%|██████████| 263/263 [00:00<00:00, 527.89it/s]


In [56]:
print(f'model: {model_name}\n{classification_report(Y_test, Y_pred)}')

model: gru-2-bidirectional-50-epochs
                       precision    recall  f1-score   support

                  DDF       0.75      0.75      0.75       234
  anatomical%location       0.79      0.68      0.73        22
               animal       0.69      0.58      0.63        19
             bacteria       0.72      0.68      0.70        84
 biomedical%technique       0.64      0.28      0.39        32
             chemical       0.61      0.48      0.54        52
   dietary%supplement       0.76      0.67      0.71        42
                 drug       0.67      0.50      0.57         8
                 food       0.00      0.00      0.00         4
                 gene       0.33      0.11      0.17         9
                human       0.74      0.75      0.75        85
           microbiome       0.74      0.80      0.77        74
statistical%technique       0.50      0.60      0.55         5

            micro avg       0.73      0.67      0.70       670
            macr

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
